<a href="https://colab.research.google.com/github/BrielleLawal/Creative-Muse/blob/main/CreativeMuse_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Installing and Importing Libraries

In [9]:
! pip install gradio

In [10]:
!pip install -q google-genai langchain langchain-google-genai langchain-core langchain-community chromadb

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown, HTML
from google.colab import userdata
from google import genai
from io import BytesIO
import gradio as gr
import base64
import time
import uuid
import os

In [16]:
import getpass
GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")
print(GOOGLE_API_KEY)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("Key succesfully configured")

Enter your Google API Key: ··········
GOOGLE_API_KEY
Key succesfully configured


#### System Prompt

In [5]:
system_prompt = """
You are CreativeMuse, a warm, imaginative, and encouraging creative assistant.
Your purpose is to help users overcome creative blocks and spark new ideas across various fields
—writing, design, music, visual arts, filmmaking, and more. When a user feels stuck, you offer unconventional prompts,
inspiring questions, or curated brainstorming techniques to get their creativity flowing again.

-Tailor your suggestions to their medium and mood, mixing practical strategies with whimsical, unexpected twists to encourage exploration.

Always be supportive, non-judgmental, and affirming—help them feel safe to experiment.
You can also suggest small creative exercises, constraints, "what if" scenarios, or draw connections between unrelated concepts to stimulate new thinking.
If a user shares part of a project, offer suggestions that expand on their work without taking it over.
"""

welcome_message = "Hey, I’m CreativeMuse—your creative companion when inspiration runs dry. Stuck on an idea? Let’s shake things up and spark something new together."

In [6]:
# Function to interact with the chatbot
def chat_bot(user_input, conversation_history=[]):
  if not conversation_history:
    return welcome_message, conversation_history
  messages = conversation_history.copy()
  messages.append({"role": "user", "content": user_input})

  # Format with system prompt
  fromatted_prompt = system_prompt + "\n\n"

  # Add conversation history
  for msg in messages:
    role = "Assistant" if msg['role'] == "'assistant" else 'user'
    fromatted_prompt += f"{role}: {msg['content']}\n\n"

  fromatted_prompt += "Assistant: "

  # Get response from model
  response = llm.invoke(fromatted_prompt)

  # Add response to history
  messages.append({"role": "assistant", "content": response.content})

  return response.content, messages

# Interactive chat loop
def interactive_chat():
  print("Welcome to CreativeMuse! type 'q' or 'quit' to end the conversation")
  conversation_history = []

  while True:
    user_input = input("\nYou: ")
    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
      print("Goodbye!")
      break

    response, conversation_history = chat_bot(user_input, conversation_history)
    print(f"\nCreativeMuse: {response}")


interactive_chat()

Welcome to CreativeMuse! type 'q' or 'quit' to end the conversation

You: q
Goodbye!


#### Chatbot with Multimodal Functionality on Gradio


In [8]:
text_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
vis_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-preview-image-generation")

# Image to Base64 Converter
def image_to_base64(image_path):
  with open(image_path, 'rb') as img:
    encoded_string = base64.b64encode(img.read())
  return encoded_string.decode('utf-8')

# Base64 to Image Converter
def base64_to_image(base64_img, filename):
  with open(filename, "wb") as f:
    f.write(base64.b64decode(base64_img))
  return filename

# Function that takes user inputs and displays it on chatUI
def query_message(message, history):
  if message["files"] is not None:
    for img in message["files"]:
      history.append({"role": "user", "content": {"path": img}})
  if message["text"] is not None:
    history.append({"role": "user", "content": message["text"]})
  return gr.MultimodalTextbox(value=None, interactive=False), history

# Function that takes user inputs, generate response and displays on chatUI
def chat_bot(history):
  messages = history.copy()

  # Format with system prompt
  system_prompt = """
                   system: You are CreativeMuse, a warm, imaginative, and encouraging creative assistant.
                   Your purpose is to help users overcome creative blocks and spark new ideas across various fields
                   — writing, design, music, visual arts, filmmaking, and more. When a user feels stuck, you offer unconventional prompts,
                   inspiring questions, or curated brainstorming techniques to get their creativity flowing again.

                   - Tailor your suggestions to their medium and mood, mixing practical strategies with whimsical, unexpected twists to encourage exploration.
                   Always be supportive, non-judgmental, and affirming—help them feel safe to experiment.
                   You can also suggest small creative exercises, constraints, "what if" scenarios, or draw connections between unrelated concepts to stimulate new thinking.
                   If a user shares part of a project, offer suggestions that expand on their work without taking it over.
                   """

  formatted_prompt = system_prompt + "\n\n"

  # Add conversation history
  for msg in messages:
    if "path" in msg["content"] and isinstance(msg["content"], dict):
      encoded_img = image_to_base64(msg["content"]["path"])
      formatted_prompt += f"{msg['role']}: data:image/jpeg.base64,{encoded_img}"
    else:
      formatted_prompt += f"{msg['role']}: {msg['content']}\n\n"

  # Get response from model
  response = vis_llm.invoke(formatted_prompt,
                            generation_config=dict(response_modalities=["TEXT", "IMAGE"]))

  # Add response to history
  history.append({"role": "assistant", "content": ""})

  for block in response.content:
    if isinstance(block, str):
      history[-1]["content"] += block
      time.sleep(0.02)
      yield history
    elif isinstance(block, dict) and block.get("image_url"):
      url = (block)["image_url"].get("url").split(",")[-1]
      image = base64_to_image(url, f"{uuid.uuid4()}.png")
      history.append({"role": "assistant", "content": {"path": image}})
      yield history

# Interface Code
with gr.Blocks() as app:
  chatbot = gr.Chatbot(
      value=[{"role": "assistant", "content": welcome_message}],
      label="CreativeMuse",
      show_label=True,
      type='messages',
      bubble_full_width=False,
      avatar_images=(None, "/content/1eb05f325ec50a15c8b045f3428d6d5e.jpg")
    )
  text_box = gr.MultimodalTextbox(
          placeholder="Enter text or upload an image...",
          container=False,
          file_types=["image"],
          file_count="multiple",
          interactive=True,
      )

  chat_msg = text_box.submit(query_message, [text_box,chatbot], [text_box, chatbot])
  bot_msg = chat_msg.then(chat_bot, chatbot, chatbot)
  bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, text_box)

app.queue()
app.launch(debug=True)


<ipython-input-8-449b2ad9aa48>:72: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9f9c4d88f982684d52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9f9c4d88f982684d52.gradio.live
